# **<center><font style="color:rgb(100,109,254)">Deteccion en tiempo real de estimacion de pose en 3D</font> </center>**

<img src="https://drive.google.com/uc?export=download&id=1CDO0KiXZEOuWc7xLEm7EFLLQf2hydCoI">

In [1]:
%pip install mediapipe

You should consider upgrading via the 'c:\Users\franc\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import math
import cv2
import numpy as np
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt

In [3]:
# Inicializando la clase de pose de mediapipe
mp_pose = mp.solutions.pose

# Seteando la funcion 'Pose'
pose = mp_pose.Pose(static_image_mode = True, min_detection_confidence = 0.3, model_complexity = 2)

# Inicialización de la clase de dibujo mediapipe, útil para la anotación.
mp_drawing = mp.solutions.drawing_utils 

In [4]:
def detectPose(image, pose, display=True):
    '''
    Esta función realiza la detección de la postura en una imagen.
    Args:
        Imagen: La imagen de entrada con una persona destacada cuyos puntos de referencia de pose necesitan ser detectados.
        pose: La función de configuración de la pose necesaria para realizar la detección de la misma.
        display: Un valor booleano que si se establece en true la función muestra la imagen de entrada original, la imagen resultante 
                 y los puntos de referencia de la pose en un gráfico 3D y no devuelve nada.
    Devuelve:
        imagen_salida: La imagen de entrada con los puntos de referencia de pose detectados dibujados.
        puntos de referencia: Una lista de puntos de referencia detectados convertidos a su escala original.

    '''
    
    # Crea una copia de la imagen de entrada
    output_image = image.copy()
    
    # Convertir la imagen de BGR a RGB
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Realiza la deteccion de poses
    results = pose.process(imageRGB)
    
    # Extrae el height y width de la imagen de entrada
    height, width, _ = image.shape
    
    # Inicializamos la lista donde almacenaremos los puntos de referencia
    landmarks = []
    
    if results.pose_landmarks:
    
        # Dibuja los puntos de referencia de la pose en la imagen de salida.
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS)
        
        # Iteramos por todos los puntos de referencia detectados
        for landmark in results.pose_landmarks.landmark:
    
            #if landmarks[landmark] == 12 or landmarks[landmark] == 11:

            # Añadimos los puntos de referencia a la lista
            landmarks.append((int(landmark.x * width), int(landmark.y * height),
                              (landmark.z * width)))

    # print('Landmarks of left sholder: ', landmarks[11])
    # print('Landmarks of right sholder: ', landmarks[12])
    print('Landmarks are: ', landmarks)
    
    # Comprueba si la imagen de entrada y la imagen resultante están especificadas para ser mostradas.
    if display:
    
        # Muestra la imagen de entrada y la imagen resultante.
        plt.figure(figsize=[22,22])
        plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Original Image");plt.axis('off');
        plt.subplot(122);plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        
        # También traza los puntos de referencia de la Pose en 3D.
        mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
        
    else:
        
        return output_image, landmarks

In [10]:
# Configurar la función Pose para el vídeo.
pose_video = mp_pose.Pose(static_image_mode = False, min_detection_confidence = 0.5, model_complexity = 1)

# Inicializa el objeto VideoCapture para leer de la webcam.
video = cv2.VideoCapture(0)

# Crear una ventana con nombre para cambiar el tamaño
cv2.namedWindow('Pose Detection in 3D', cv2.WINDOW_NORMAL)

# Seteando las dimensiones de la camara
video.set(3,1280)
video.set(4,960)

# varialbe para tomar tiempo por cada frame
time1 = 0

while video.isOpened():
    
    # Leer un frame
    ok, frame = video.read()
    
    # Si el frame no esta Ok
    if not ok:
        break
    
    # Voltea el marco horizontalmente para una visualización natural (selfie-view).
    frame = cv2.flip(frame, 1)
    
    # Obtener la anchura y la altura del frame
    frame_height, frame_width, _ =  frame.shape
    
    # Cambia el tamaño del cuadro manteniendo la relación de aspecto.
    frame = cv2.resize(frame, (int(frame_width * (640 / frame_height)), 640))
    
    # Realice la detección de puntos de referencia de la pose.
    frame, _ = detectPose(frame, pose_video, display = False)
    
    # Setear el tiempo del frame a la hora actual
    time2 = time()
    
    # Mostrar el frame
    cv2.imshow('Pose Detection in 3D', frame)
    
    # capturar que tecla se presiona
    k = cv2.waitKey(1) & 0xFF
    
    # si pulsa 'ESC' cerrar ventana
    if(k == 27):
        break

# Liberar el objeto VideoCapture.
video.release()

# Cerrar la ventana
cv2.destroyAllWindows()

Landmarks are:  [(635, 335, -973.3822696208954), (662, 293, -922.9217247962952), (677, 294, -923.0771225094795), (690, 294, -923.1613612174988), (608, 290, -920.9312379956245), (589, 288, -920.2969740629196), (574, 287, -920.4192997813225), (708, 314, -544.0518328249454), (553, 304, -530.9541707932949), (664, 382, -832.1716289520264), (598, 378, -828.6415323615074), (826, 564, -409.0191492140293), (416, 504, -328.1464654505253), (871, 813, -658.5321353673935), (204, 589, -778.9269372224808), (841, 1013, -1206.8159297704697), (10, 571, -1741.715596318245), (852, 1091, -1336.6873490810394), (-61, 569, -1916.2363772392273), (811, 1075, -1377.49127805233), (-36, 538, -1965.7813428640366), (797, 1047, -1249.5105196237564), (12, 557, -1801.8506484031677), (713, 1045, -41.13752916455269), (441, 1035, 45.59100279584527), (695, 1452, -4.686275293584913), (444, 1444, 147.8510973751545), (697, 1829, 558.4913164973259), (450, 1811, 566.7265481650829), (705, 1889, 582.2807884812355), (445, 1866, 59